# Demonstration 1: Open Field

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sisyphus import ValueIteration
%matplotlib inline

## Value Iteration

In [ ]:
from sisyphus.envs import OpenField

fig, axes = plt.subplots(1,3,figsize=(12,4))
gym = OpenField()

for ax, beta in zip(axes, [10,0,-10]):
    qvi = ValueIteration(policy='softmax', beta=beta)
    qvi = qvi.fit(gym)
    sns.heatmap(qvi.V.reshape(gym.shape), center=0, vmin=-10, vmax=10, ax=ax,
                xticklabels=[], yticklabels=[], cbar=False)
    
plt.tight_layout()
plt.savefig('figures/01_field.png', dpi=180)